In [1]:
import os
import re

files = os.listdir('./Novels')
print(files)

['1569.txt', '26814.txt', '27856.txt', '41803.txt', '43793.txt', '44374.txt', '45038.txt', '46156.txt', '47874.txt', '48882.txt', '52871.txt', '53808.txt', '56007.txt', '56652.txt', '59181.txt', '60174.txt', '61925.txt', '65182.txt', '67454.txt', '67586.txt', '67646.txt', '67979.txt', '68222.txt', '68273.txt', '68363.txt', '68833.txt', '68973.txt', '69071.txt', '69413.txt', '69569.txt', '70010.txt', '70290.txt', '70479.txt', '70696.txt', '70727.txt', '70837.txt', '70876.txt', '71024.txt', '71205.txt', '71376.txt', '71761.txt', '71769.txt', '71978.txt', '73080.txt']


In [2]:
def count_syllables(word):
    vowels = "aeiouáéíóú"
    word = word.lower().strip()
    if word.endswith('e') or word.endswith('s'):
        word = word[:-1]
    
    syllable_count = 0
    previous_char_was_vowel = False
    
    for char in word:
        if char in vowels:
            if not previous_char_was_vowel: # Si ahora tengo una vocal y la anterior no fue vocal, me agrega una sílaba
                syllable_count += 1
            previous_char_was_vowel = True # Seteo que la previa fue vocal
        else:
            previous_char_was_vowel = False 

    return syllable_count

In [3]:
# Testeo que esté bien
print(count_syllables('creeps'))
print(count_syllables('devotion'))

1
3


In [4]:
def difficult_word(word):
    n_syllabes = count_syllables(word)
    if n_syllabes > 2:
        difficult_word = 1
    else:
        difficult_word = 0
    return difficult_word

In [5]:
print(difficult_word('cat'))
print(difficult_word('ubiquitous'))

0
1


In [6]:
root = './Novels/'
number_sentences = []
number_words = []
number_characters = []
number_syllables = []
difficult_words = []
for file in files:
    file_path = root + file
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            file_content = f.read()
            replaced_text = file_content.replace('!', '.').replace('?', '.').replace('\n', ' ')
            sentences = replaced_text.split('.')
            number_sentences.append(len(sentences))
            #words = replaced_text.split(' ') # Sin regexp
            words = re.findall(r'\b\w+\b', replaced_text) # Con regexp
            number_words.append(len(words))
            character_count = 0
            syllable_count = 0
            difficult_word_count = 0
            for word in words:
                character_count += len(word)
                syllable_count += count_syllables(word)
                difficult_word_count += difficult_word(word)
            number_characters.append(character_count)
            number_syllables.append(syllable_count)
            difficult_words.append(difficult_word_count)

In [7]:
for i, file in enumerate(files):
    print(f"File: {file} - Sentences: {number_sentences[i]} - Words: {number_words[i]} - Characters: {number_characters[i]} - Syllables: {number_syllables[i]} - Difficult Words: {difficult_words[i]}")

File: 1569.txt - Sentences: 4946 - Words: 105311 - Characters: 456532 - Syllables: 133122 - Difficult Words: 9620
File: 26814.txt - Sentences: 2286 - Words: 37844 - Characters: 159761 - Syllables: 46303 - Difficult Words: 4282
File: 27856.txt - Sentences: 4445 - Words: 60987 - Characters: 260868 - Syllables: 78097 - Difficult Words: 5604
File: 41803.txt - Sentences: 8669 - Words: 125379 - Characters: 532929 - Syllables: 152077 - Difficult Words: 8119
File: 43793.txt - Sentences: 2477 - Words: 42596 - Characters: 174078 - Syllables: 50931 - Difficult Words: 2552
File: 44374.txt - Sentences: 3422 - Words: 52573 - Characters: 228958 - Syllables: 65420 - Difficult Words: 4551
File: 45038.txt - Sentences: 10966 - Words: 191783 - Characters: 781077 - Syllables: 228541 - Difficult Words: 11302
File: 46156.txt - Sentences: 8656 - Words: 219232 - Characters: 891472 - Syllables: 262476 - Difficult Words: 12511
File: 47874.txt - Sentences: 8563 - Words: 170515 - Characters: 708060 - Syllables: 21

In [8]:
ars = []
fki = []
d_c = []
for i in range(0, len(number_words)):
    cpw = number_characters[i]/number_words[i]
    wps = number_words[i]/number_sentences[i]
    spw = number_syllables[i]/number_words[i]
    diff_words = difficult_words[i]/number_words[i]
    ars_val = 4.71*cpw + 0.5*wps - 21.43
    ars.append(ars_val)
    fki_val = 0.39*wps + 11.8*spw - 15.59
    fki.append(fki_val)
    d_c_val = 15.79*diff_words + 0.0496*wps
    d_c.append(d_c_val)

In [9]:
for i, file in enumerate(files):
    print(f"File: {file} - Automated Readability Score: {ars[i]} - Flesch-Kinkaid Index: {fki[i]} - Dale-Chall Readability Score: {d_c[i]}")

File: 1569.txt - Automated Readability Score: 9.634321886485044 - Flesch-Kinkaid Index: 7.630137346586292 - Dale-Chall Readability Score: 2.498483434332715
File: 26814.txt - Automated Readability Score: 6.7309231989630405 - Flesch-Kinkaid Index: 5.303895483627912 - Dale-Chall Readability Score: 2.6077303831551233
File: 27856.txt - Automated Readability Score: 5.5769043613879035 - Flesch-Kinkaid Index: 4.871449179408923 - Dale-Chall Readability Score: 2.13144824621257
File: 41803.txt - Automated Readability Score: 5.821520801742587 - Flesch-Kinkaid Index: 4.3632091701904905 - Dale-Chall Readability Score: 1.7398524106181026
File: 43793.txt - Automated Readability Score: 6.416763880247814 - Flesch-Kinkaid Index: 5.225650105868343 - Dale-Chall Readability Score: 1.7989579942931064
File: 44374.txt - Automated Readability Score: 6.763900525703576 - Flesch-Kinkaid Index: 5.085169513314728 - Dale-Chall Readability Score: 2.12888343126531
File: 45038.txt - Automated Readability Score: 6.496912

In [10]:
# Una función para encontrar el titulo usando regex
def find_title(filecontent):
    match = re.search(r'Title:\s*(.+?)\n', filecontent)
    if match:
        return match.group(1).strip()
    else:
        return "Title not found."

In [11]:
def measure_complexity(filepath):
    if filepath.endswith('.txt'):
        with open(filepath, 'r', encoding='utf-8') as f:
            file_content = f.read()
            title = find_title(file_content)
            replaced_text = file_content.replace('!', '.').replace('?', '.').replace('\n', ' ')
            sentences = [s.strip() for s in replaced_text.split('.') if s.strip()] # Filtra oraciones vacías
            sentence_count = len(sentences)
            if sentence_count == 0:
                return "The file does not contain any sentences."
            #words = replaced_text.split(' ')
            words = re.findall(r'\b\w+\b', replaced_text) # Usa regex para definir palabras
            word_count = len(words)
            if word_count == 0:
                return "The file does not contain any words."
            character_count = 0
            syllable_count = 0
            difficult_word_count = 0
            for word in words:
                character_count += len(word)
                syllable_count += count_syllables(word)
                difficult_word_count += difficult_word(word)
            cpw = character_count/word_count
            wps = word_count/sentence_count
            spw = syllable_count/word_count
            diff_words = difficult_word_count/word_count
            ars = 4.71*cpw + 0.5*wps - 21.43
            fki = 0.39*wps + 11.8*spw - 15.59
            d_c = 15.79*diff_words + 0.0496*wps
    #return title, ars, fki, d_c
    return (f"Title: {title} - Automated Readability Score: {ars:.3f} - "
                f"Flesch-Kincaid Index: {fki:.3f} - "
                f"Dale-Chall Readability Score: {d_c:.3f}")
    

In [14]:
measure_complexity(root + files[10])

'Title: Laakson lilja: Maaseutuelämän kuvaus - Automated Readability Score: 17.748 - Flesch-Kincaid Index: 14.703 - Dale-Chall Readability Score: 5.644'